In [1]:
import sys
import os
import sqlite3

model_name = 'gemini-2.5-pro'

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name=model_name)

# Load the PRD from Day 1
with open("../documents/template_PRD.md", "r", encoding="utf-8") as f:
    prd_content = f.read()

if not prd_content:
    print("Warning: Could not load template_PRD.md. Lab may not function correctly.")

2025-10-02 11:27:45,875 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


In [15]:
schema_prompt = f"""You are a database administrator and SQLite expert. 
- Based on the following Product Requirements Document (PRD), generate a SQLite schema that includes tables, columns, data types, primary keys, foreign keys, and necessary constraints to effectively represent all of the data described in the PRD.
- Ensure that the schema is normalized to at least the third normal form (3NF) to minimize redundancy and ensure data integrity.
- Design the schema and include relationships between them. If there is a need for additional tables to represent many-to-many relationships or other entities, include those as well.
- Extraneous information such as preferred communication methods, office hours, or other non-essential user preferences should not be included in the schema.
- Use appropriate SQLite data types for each column based on the data described in the PRD. 
- Provide the SQLite schema in a single code block without any additional explanation or commentary.
- Ensure that the timestamp fields use the appropriate SQLite data type and default values.
- Ensure that related fields use on delete cascade where appropriate.
- Only focus on generating data for version 1.0 as described in the PRD.
- If a previous schema and seed generation comments are available, ensure that the new schema improves upon it by adding or removing tables, elements, or relationships based on the PRD and seed data generation comments.


<PRD context>
{prd_content}
</PRD context>

<Previous schema>
{cleaned_schema if 'cleaned_schema' in locals() else 'Previous schema not available.'}
</Previous schema>

<Seed generation>
{cleaned_seed_data if 'cleaned_seed_data' in locals() else 'Seed data not available.'}
</Seed generation>
"""

# enhanced_prompt = prompt_enhancer(schema_prompt)

print("Generating SQL Schema...")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)

    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    # print(cleaned_schema)
    
    with open("../documents/db_schema.sql", "w", encoding="utf-8") as f:
         f.write(cleaned_schema)

else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

Generating SQL Schema...


In [16]:
with open("../documents/db_schema.sql", "r", encoding="utf-8") as f:
    cleaned_schema = f.read()

seed_data_prompt = f'''You are a senior database administrator and SQLite expert. Using the following SQLite schema and Product Requirements Document (PRD), generate realistic and coherent seed data for all tables, ensuring that all foreign key and relational constraints are respected.
- Populate all tables with appropriate data types and formats, using plausible values that reflect a real-world scenario for a tech company's onboarding system.
- Ensure that users reference valid roles and teams, and that onboarding tasks reference valid users as creators.
- Include at least 2 onboarding workflows, and assign tasks to workflows using the workflow_tasks table, with meaningful sequence_order values.
- Populate user_onboarding_progress with at least 10 records, showing a mix of statuses and realistic dates, and referencing valid users, tasks, and workflows.
- All data should be internally consistent and reflect plausible onboarding scenarios for a tech company.
- Use appropriate INSERT statements for each table, and provide all statements in a single code block with no extra explanation. Do not include markdown or other formatting.
- No additional tables should be created in the seed data.
- If certain fields are not specified in the schema, use reasonable defaults (e.g., current date for created_at).
- If a table is missing that would logically be needed to represent the data, generate a comment indicating the omission at the end of the file, but do not create the table.
- If a table appears to be unnecessary based on the PRD, generate a comment indicating the redundancy at the end of the file, but do not create the table.
- Use additional context from previous seed data generation comments to inform your seed data (if available).
- Only focus on generating data for version 1.0 as described in the PRD.
    - If features or tables from versions beyond 1.0 are present in the schema, add a comment at the end of the seed data indicating that they should not be included in the schema and do not generate seed data for them.

<Previous seed data generation comments>
{cleaned_seed_data if 'cleaned_seed_data' in locals() else 'N/A'}
</Previous seed data generation comments>

<SQLite Schema>
{cleaned_schema}
</SQLite Schema>

<Product Requirements Document>
{prd_content}
</Product Requirements Document>
'''

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    # print(cleaned_seed_data)
    
    # Save the cleaned seed data
    # save_artifact(cleaned_seed_data, 'artifacts/seed_data.sql', overwrite=True)
    with open("../documents/seed_data.sql", "w", encoding="utf-8") as f:
         f.write(cleaned_seed_data)
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---
